<a href="https://colab.research.google.com/github/SWEN90006/tutorials/blob/main/SWEN90006_Tutorial_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SWEN90006 Tutorial 6

## Introduction
The aim of this tutorial is for you to familiarise yourself with the
test oracles and random testing. This tutorial will help to gain an
understanding of how to specify a test oracle to decide whether an
arbitrary test case is correct, incorrect or undecidable, as well as how
to randomly generate test cases that fit an operational profile.

## Working With the Program

### First Program

We shall start this tutorial by using a rather simple example to
illustrate the concepts, techniques and issues faced when working with
oracles. This is followed by applying these techniques to the root
finding program from tutorial 4 to gain some practice with practical
examples.

This *toLower* is an application which takes a string on standard input
and puts a corresponding string on standard output, with all upper case
letters changed to the matching lower case letters (i.e. 'A' to 'a', 'B'
to 'b', etc.). Strings may consist of all ASCII characters between 32
and 126 inclusive; characters outside this range are control characters
and will cause an error message.

### Prepare the Java Kernel
Since Java is not natively supported by Colab, we need to run the following code to enable Java kernel on Colab.

1. Run the cell bellow (click it and press Shift+Enter),
2. Refresh the Notebook (F5)
3. Change the kernel to Java (Runtime -> Change Runtime Type -> Java)

In [ ]:
!wget https://github.com/SpencerPark/IJava/releases/download/v1.3.0/ijava-1.3.0.zip
!unzip ijava-1.3.0.zip
!python install.py --sys-prefix

In [ ]:
%%loadFromPOM

<dependency>
  <groupId>junit</groupId>
  <artifactId>junit</artifactId>
  <version>4.13.2</version>
</dependency>

The following is a basic Java implementation of `ToLower`.

In [ ]:
public class ToLower {

    static final int MAX_STRING = 80;
    
    public static char[] toLower(String input)
    {
        //Scanner scan = new Scanner(System.in);
        char[] string = subString(input, 0, MAX_STRING).toCharArray();
        for (int i = 0; i < string.length - 1; i++) {
            if (string[i] < 32 || string[i] == 127) {
                System.out.println("Illegal character found.\n");
                return "Illegal".toCharArray();
            }
            
            if (string[i] >= 65 || string[i] <= 90) {
                string[i] = (char) (string[i] + 'a' - 'A');
            }
        }
        return string;
    }
    
    public static void main(String[] args) {
        System.out.println(toLower("€"));
    }
    
    public static String subString(String myString, int start, int length) {
        return myString.substring(start, Math.min(start + length, myString.length()));
    }
    
}

In [ ]:
import org.junit.Assert;
import java.util.Arrays;
import java.util.Collection;

import org.junit.Test;
import org.junit.runner.JUnitCore;
import org.junit.runner.Result;
import org.junit.runner.RunWith;
import org.junit.runner.notification.Failure;
import org.junit.runners.Parameterized;

import junit.framework.TestCase;

@RunWith(Parameterized.class)
public class TestToLower extends TestCase {
    @Parameterized.Parameter(0)
    public String actual;
    @Parameterized.Parameter(1)
    public char[] expected;
    
    @Parameterized.Parameters(name = "{index}: actual: {0} expected:{1}")
    public static Collection<Object[]> data() {

        
        Object[][] data = new Object[][]{
            // Your Test cases start here

            {"KKK", new char[] {'k', 'k', 'k'}},
            // Your Test cases end here
            };
        return Arrays.asList(data);
    }
    
    @Test
    public void testLower() {
        Assert.assertArrayEquals(expected, ToLower.toLower(actual));
    }
    
    public static void main(String[] args) {

    }
}

In [ ]:
Result result = JUnitCore.runClasses(TestToLower.class);
for (Failure failure : result.getFailures()) {
     System.out.println(failure.toString());
}
System.out.println(String.format("Total run count: %s, Failed run count: %s", result.getRunCount(), result.getFailureCount()));

testLower[0: actual: KKK expected:[C@656a01f0](REPL.$JShell$26$TestToLower): arrays first differed at element [2]; expected:<k> but was:<K>
Total run count: 1, Failed run count: 1


### Second Program

The idea behind the algorithm for finding roots is to look at the
interval $[Lower,Upper]$ and bisect it (hence the name of the algorithm)
and find the midpoint of the interval $x_r$. If we know that $Lower$ is
negative, and $Upper$ is positive then there must be root in the
interval, provided that the function is continuous. If the value of $f$
at $x_r$ is positive then the root must be in the interval
$[Lower,x_r]$. If the value of $f$ at $x_r$ is negative then the root
must be in the interval $[x_r,Upper]$. The algorithm should converge to
the root because the length of the interval is getting smaller every
time (in fact the length of the interval is halved every time).

The following is a basic Java implementation of `BiSection`.

In [ ]:
public class Bisection{

    static final int MAX_INT = 65535;
    
    static double bisection(double lower, double upper, double error, int max) {
        
        double sign = 0.0; /* Test for the sign of the midpoint xr. */
        double ea = MAX_INT; /* Calculated error value. */
        double xrold = 0.0; /* Previous estimate. */
        double xr = 0.0; /* Current x estimate for the root. */
        double fr = 0.0; /* Current value of f. */
        double fl = 0.0; /* Value of f at the lower end of the interval. */
        int iteration = 0; /* For keeping track of the number of iterations. */
        
        fl = func(lower);
        while ((ea > error && iteration < max)) {
            
            /* Start by memorising the old estimate in xrold and then calculate
            the new estimate and store in fr */
            xrold = xr;
            xr = (lower + upper) / 2;
            fr = func(xr);
            iteration++;
            /* Estimate the percentage error and store in ea. */
            if (xr != 0) {
                ea = Math.abs((xr - xrold)/xr) * 100;
            }
            
            /* To know whether fr has the same sign as f(Lower) or f(Upper) is easy:
            we know that f(Lower) is negative and we know that f(Upper) is positive.
            Multiple fr by f(Lower) and if the result is positive then fr must be
            negative. If the result is negative then fr must be positive. */
            
            sign = func(lower) * fr;
            if (sign < 0)
                upper = xr;
            else if (sign > 0)
                lower = xr;
            else
                ea = 0;
            System.out.println(String.format("iteration %d = (%f, %f, %f, %f, %f)\n", iteration, lower, upper, xr, ea, sign));
        }
        return xr;
    }
    
    static double func(double x) {
        return x - 2;
    }
}

In [ ]:
import java.util.Arrays;
import java.util.Collection;

import org.junit.Test;
import org.junit.runner.JUnitCore;
import org.junit.runner.Result;
import org.junit.runner.RunWith;
import org.junit.runner.notification.Failure;
import org.junit.runners.Parameterized;

import junit.framework.TestCase;

@RunWith(Parameterized.class)
public class TestBisection extends TestCase {
    
    @Parameterized.Parameter(0)
    public double lower;
    @Parameterized.Parameter(1)
    public double upper;
    @Parameterized.Parameter(2)
    public double error;
    @Parameterized.Parameter(3)
    public int max;
    @Parameterized.Parameter(4)
    public double results;

    @Parameterized.Parameters(name = "{index}: lower: {0} upper:{1} error:{2} iterations:{3} results:{5}")
    public static Collection<Object[]> data() {
        Object[][] data = new Object[][]{
            // Your Test cases start here
            // Please follow the pattern: lower, upper, error, iterations, expected results
            {-1.0, 7.0, 1, 10, 2.0},
            {-1.0, 7.0, 1, 10, 2.0}
            // Your Test cases end here
            };
        return Arrays.asList(data);
    }

    @Test
    public void testBisection() {
        assertEquals(results, Bisection.bisection(lower, upper, error, max));
    }
}

In [ ]:
Result result = JUnitCore.runClasses(TestBisection.class);
for (Failure failure : result.getFailures()) {
     System.out.println(failure.toString());
}
System.out.println(String.format("Total run count: %s, Failed run count: %s", result.getRunCount(), result.getFailureCount()));

iteration 1 = (-1.000000, 3.000000, 3.000000, 100.000000, -3.000000)

iteration 2 = (1.000000, 3.000000, 1.000000, 200.000000, 3.000000)

iteration 3 = (1.000000, 3.000000, 2.000000, 0.000000, -0.000000)

iteration 1 = (-1.000000, 3.000000, 3.000000, 100.000000, -3.000000)

iteration 2 = (1.000000, 3.000000, 1.000000, 200.000000, 3.000000)

iteration 3 = (1.000000, 3.000000, 2.000000, 0.000000, -0.000000)

Total run count: 2, Failed run count: 0


## Your Tasks

Repeat these tasks for both programs.

### Task 1
Standard analysis: Determine the input/output domains and the
input/output conditions.

### Task 2
Using the specification and the input domain, write an automated test oracle to determine if an arbitrary test input is correct, incorrect or otherwise undecidable.

### Task 3
Determine an automated means for generating random test cases by selection points from the input set.
